In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['posts']
posts_collection = db['posts']
posts_collection.drop()




In [ ]:
from itertools import islice
import json
from datetime import datetime

with open ('data.jsonl', 'r') as file:
    for line in islice(file,None):
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [ ]:
#1 Insertar 5 nuevos documentos de usuario con datos únicos
from itertools import islice
import json
from datetime import datetime


with open ('datos_reales.jsonl', 'r') as file:
    for line in islice(file,None):
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [ ]:
#2 Buscar todos los usuarios con edad mayor a 25.

cursore = posts_collection.find({'user_age': {'$gt': 25}},{'_id': 0})
for doc in cursore:
    print(doc)

In [ ]:
#list(posts_collection.find({}, {'user': 1, "user_email": {'$eq': "tmartin@example.org"}, '_id': 0}))
# 3. Buscar un usuario por dirección de correo electrónico

cursore = posts_collection.find({'user_email': {'$eq': 'tmartin@example.org'}},{'_id': 0})
for doc in cursore:
    print(doc)

In [20]:
# 4. Actualizar el contenido de la publicación de un usuario específico.
from bson import ObjectId

posts_collection.update_one({'_id': ObjectId('65f1255d606c54a96079555d')}, {'$set': {'post.content': 'Dylan estuvo aqui'}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
#2.1 Buscar todos los usuarios cuyo nombre de usuario contenga la palabra S̈mith.̈ (Expresiones regulares)
puntero=posts_collection.find({ 'user': { '$regex': 'Smith' } })
for doc in puntero:
    print(doc)

In [ ]:
# 2.2 Buscar publicaciones creadas después de una fecha específica (por ejemplo, 2000-01-01).
start_date = datetime(2000, 1, 1)

puntero=posts_collection.find({"timestamp": {"$gt": start_date}})
for doc in puntero:
    print(doc)

In [ ]:
#3.1 Buscar usuarios que tengan más de 20 años pero menos de 30

cursore = posts_collection.find({'user_age': {'$gt': 20,'$lt':30}},{'_id': 0})
for doc in cursore:
    print(doc)

In [ ]:
# 3.2 Recuperar solo el nombre de usuario y la edad del usuario de todos los documentos. (Proyección)

list(posts_collection.find({}, {'user': 1, "age": 1, '_id': 0}))

In [ ]:
# 4.1 Ordenar todos los usuarios por edad en orden descendente.

#from pymongo import MongoClient

cursore = posts_collection.find({}).sort('user_age', 1)
for doc in cursore:
    print(doc)
                                            

In [ ]:
#4.2 Buscar los 3 usuarios principales con las publicaciones más recientes (en función de la marca de tiempo).
from datetime import datetime

fecha_actual = datetime.now()

cursore = posts_collection.find({'post.timestamp': {'$lte': fecha_actual}},{'_id': 0}).sort('post.timestamp', -1).limit(3)
for doc in cursore:
    print(doc)


In [ ]:
# 5.1 Utilizar el pipeline de agregación para calcular el número total de usuarios.



In [ ]:
#5.4 Calcular la longitud promedio de los comentarios en todos los comentarios.

#"$unwind": "$comments" descomprime el campo comments, haceindo mas facil despues trabajar con ese campo
#{"$strLenCP": "$comments.comment.text"} accede al comentario que le hicieron al post
cursore = list(posts_collection.aggregate([{"$unwind": "$comments"},{"$group": {"_id": None, "avg_length": {"$avg": {"$strLenCP": "$comments.comment.text"}}}}],session=None))


# Obtener el resultado de la agregación
longitud_promedio = cursore[0]["avg_length"] if cursore else 0

# Imprimir la longitud promedio de los comentarios
print("Longitud promedio de los comentarios:", longitud_promedio)


In [ ]:
#6.2 Establecer el campo ”contenido” de una publicación específica en una cadena vacía.

cursore = posts_collection.find_one_and_update({'post.content': {'$eq': 'los reales nunca moriran brrrrrrr.'}}, {'$set': {'post.content': ''}}, return_document=True)